Data aumentation

In [3]:
import albumentations as A
import cv2
import os
import random

# Directorios de entrada y su número correspondiente de aumentaciones
input_directories = {
    #"dia": 1,
    "doctor": 7,
    #"domestico": 1,
    #"dulce": 1,
    #"granja": 9 ,
    #"mecanico": 1,
    #"montaña": 23,
    #"noche": 30,
    "playa": 27,
    #"saludable": 1,
    "salvaje": 7
}

# Definir las transformaciones que deseas aplicar
transform = A.Compose([
    A.RandomRotate90(),  # Rotación aleatoria de 90 grados
    A.HorizontalFlip(p=0.5),  # 50% de probabilidad de hacer flip horizontal
    A.VerticalFlip(p=0.5),  # 50% de probabilidad de hacer flip vertical
    A.Transpose(),

    # Ajustes de color y brillo
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.HueSaturationValue(),
        A.RGBShift(),
    ], p=0.5),

    # Aplicar desenfoques aleatorios
    A.OneOf([
        A.Blur(blur_limit=4),
        A.MotionBlur(blur_limit=4),
        A.MedianBlur(blur_limit=4),
    ], p=0.5),

    # Agregar reducciones y recortes de tamaño
    A.OneOf([
        A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.5),  # Recorte y redimensionamiento aleatorio
        A.CenterCrop(height=256, width=256, p=0.5),  # Recorte centrado
    ], p=0.5),

    # Asegurarse de redimensionar todas las imágenes a un tamaño estándar
    A.Resize(height=256, width=256),
])


# Función para aumentar y guardar las imágenes
def augment_and_save(image_path, transform, output_dir, num_augmented):
    # Cargar imagen
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Nombre base del archivo de imagen
    filename = os.path.splitext(os.path.basename(image_path))[0]

    # Generar y guardar imágenes aumentadas
    for i in range(num_augmented):
        # Aplicar transformación
        augmented = transform(image=image)
        augmented_image = augmented["image"]

        # Guardar la imagen aumentada con un nombre único
        new_filename = f"{filename}_aug_{i+1}.jpg"
        output_path = os.path.join(output_dir, new_filename)

        # Convertir la imagen de nuevo a BGR para guardar con OpenCV
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, augmented_image_bgr)
        print(f"Imagen aumentada guardada en: {output_path}")

# Ruta base para los directorios de entrada y salida
base_input_dir = '/home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation'  # Cambiar a tu ruta de entrada
base_output_dir = '/home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/nuevasImagenes'  # Cambiar a tu ruta de salida

# Procesar cada directorio
for directory, num_augmented in input_directories.items():
    input_dir = os.path.join(base_input_dir, directory)
    output_dir = os.path.join(base_output_dir, directory)

    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    print(f"Procesando directorio: {input_dir} con {num_augmented} aumentaciones por imagen")

    # Aplicar aumentación a cada imagen en el directorio
    for image_filename in os.listdir(input_dir):
        image_path = os.path.join(input_dir, image_filename)
        if os.path.isfile(image_path):
            augment_and_save(image_path, transform, output_dir, num_augmented)


/home/andres/ambientesVirtuales/PruebasML/lib/python3.10/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `dict[str, any]` but got `UniformParams` with value `UniformParams(noise_type=...6, 0.0784313725490196)])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/andres/ambientesVirtuales/PruebasML/lib/python3.10/site-packages/albumentations/augmentations/blur/functional.py:149: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (3, 4) automatically adjusted to (3, 5).
  result = _ensure_odd_values(result, info.field_name)


Procesando directorio: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/doctor con 7 aumentaciones por imagen
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_1.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_2.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_3.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_4.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_5.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/doctor/manguito_aug_6.jpg
Imagen aumentada guardada en: /home/andres/ambientesVirtuales/PruebasML/Practicas/dataAumentation/new/d